In [23]:
import pytorch_apis
from pytorch_apis import matrixMul
import torch
import torch.nn as nn

class CustomLinear(nn.Module):
    def __init__(self, in_features, out_features, bias=True):
        super(CustomLinear, self).__init__()
        self.weight = nn.Parameter(torch.randn(out_features, in_features)) 
        if bias:
            self.bias = nn.Parameter(torch.randn(out_features)) 
        else:
            self.bias = None

    def forward(self, x):
        output = matrixMul(x, self.weight.t())  # [batch_size, out_features]
        # output = torch.matmul(x, self.weight.t())  # [batch_size, out_features]
        if self.bias is not None:
            output += self.bias
        return output


In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

class LeNet300100(nn.Module):
    def __init__(self):
        super(LeNet300100, self).__init__()
        # self.fc1 = nn.Linear(28*28, 300)
        # self.fc2 = nn.Linear(300, 100)
        # self.fc3 = nn.Linear(100, 10)
        self.fc1 = CustomLinear(28*28, 300)
        self.fc2 = CustomLinear(300, 100)
        self.fc3 = CustomLinear(100, 10)
        
        self.relu = nn.ReLU()

        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = x.view(-1, 28*28)

        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))

        x = self.softmax(self.fc3(x))

        return x

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

model = LeNet300100()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

def train(model, trainloader, optimizer, criterion, epochs=5):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(trainloader):
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 100 == 99:  # print every 100 mini-batches
                print(f'Epoch {epoch + 1}, Batch {i + 1}, Loss: {running_loss / 100}')
                running_loss = 0.0

def test(model, testloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in testloader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy of the model on the test images: {100 * correct / total:.2f}%')

train(model, trainloader, optimizer, criterion, epochs=5)
test(model, testloader)
